<a href="https://colab.research.google.com/github/thoriqakhdn/KNN/blob/main/PKB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from math import sqrt
from math import pow

#Membaca data test dari file traintest
data = pd.read_excel('https://github.com/thoriqakhdn/KNN/blob/main/traintest.xlsx?raw=true', index_col='id')
test_data = pd.read_excel('https://github.com/thoriqakhdn/KNN/blob/main/traintest.xlsx?raw=true', sheet_name='test', index_col='id')[['x1', 'x2', 'x3']]

#Membuat K-Fold Cross Validation
validation_data = data.sample(frac=.2)
train_data = data.drop(validation_data.index)

#training dataset 
train_data_x = train_data[['x1', 'x2', 'x3']].values.tolist()
train_data_y = train_data['y'].values.tolist()

#training validation dataset
validation_data_x = validation_data[['x1', 'x2', 'x3']].values.tolist()
validation_data_y = validation_data['y'].values.tolist()

#Eucledian Distance untuk menghitung jarak
def eucl_dist(x1, x2):
    distance = ((x1[0] - x2[0])**2) + ((x1[1] - x2[1])**2) + ((x1[2] - x2[2])**2)
    return sqrt(distance)

#Mengklasifikasikan y
def klasifikasi(train_list):
    y_0 = 0
    y_1 = 0
    i = 0
    
    #proses perulangan dari awal list sampai akhir untuk mengklasifikasikan y dari setiap id
    while i < len(train_list):
        val = train_list[i][1]
        if val == 0:
            y_0 += 1
        elif val == 1:
            y_1 += 1
        i += 1
    
    if y_0 > y_1:
        return 0
    elif y_0 < y_1:
        return 1

#Mencari Neighbours
def neighbours(train_data_x, train_data_y, titik_baru, k):
    train_list = []
    i = 0
    while i != len(train_data_x):
        train_list.append([eucl_dist(train_data_x[i], titik_baru), train_data_y[i]])
        i += 1
    train_list.sort(key=lambda x: x[0])
    return klasifikasi(train_list[:k])

#Menguji hasil aktual dari prediksi
def prediction(k, train_data_x, train_data_y, validation_data_x, nilai_y=None):
    prediction_x_actual = []
    banyak_data = len(validation_data_x)
    for i in range (0, banyak_data, 1):
        if nilai_y == None:
            prediction_x_actual.append(neighbours(train_data_x, train_data_y, validation_data_x[i], k))
        else:
            prediction_x_actual.append([neighbours(train_data_x, train_data_y, validation_data_x[i], k), validation_data_y[i]])
    return prediction_x_actual

#Confusion matrix
def confusion_matrix(train_data_x, train_data_y, validation_data_x, validation_data_y):
    nilai_k = range(1, 10, 2)
    hasil = []
    for k in nilai_k:
        TP = 0
        TN = 0
        FP = 0
        FN = 0
        prediction_actual = prediction(k, train_data_x, train_data_y, validation_data_x, validation_data_y)
        i = 0
        while i != len(prediction_actual):
            if prediction_actual[i][0] == 1 and prediction_actual[i][1] == 1:
                TP += 1
            elif prediction_actual[i][0] == 1 and prediction_actual[i][1] == 0:
                FP += 1
            elif prediction_actual[i][0] == 0 and prediction_actual[i][1] == 1:
                FN += 1
            elif prediction_actual[i][0] == 0 and prediction_actual[i][1] == 0:
                TN += 1
            i += 1
        accuracy = (TP+TN)/(TP+TN+FP+FN)
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        f1_score = (2*precision*recall)/(precision+recall)
        hasil.append([k, accuracy, f1_score])
    x = 0
    while x < len(hasil):
        print(hasil[x])
        x += 1 
    hasil.sort(key=lambda x: (-x[1], x[2]))
    print("K terbaik adalah", hasil[0][0], "dengan hasil F-1 ", hasil[0][2])
    return hasil[0]

def main(data, test_data, train_data_x, train_data_y, validation_data_x, validation_data_y):
    #
    data_x = data[['x1', 'x2', 'x3']].values.tolist()
    data_y = data['y'].values.tolist()
    test = test_data.values.tolist()
    
    #Testing Training Model
    k_terbaik = confusion_matrix(train_data_x, train_data_y, validation_data_x, validation_data_y)[0]
    result = prediction(k_terbaik, data_x, data_y, test)
    
    #Menyimpan hasil data hasil Testing ke variable untuk dioutput kefile xlsx
    test_result = test_data
    test_result['y'] = result
    print(test_result)
    test_result.to_excel('Hasil_testing.xlsx', sheet_name='test')
    return;

datafinal = main(data, test_data, train_data_x, train_data_y, validation_data_x, validation_data_y)
datafinal 

[1, 0.7627118644067796, 0.8333333333333333]
[3, 0.6779661016949152, 0.7865168539325843]
[5, 0.7966101694915254, 0.8695652173913044]
[7, 0.7796610169491526, 0.8631578947368421]
[9, 0.7796610169491526, 0.8659793814432989]
K terbaik adalah 5 dengan hasil F-1  0.8695652173913044
     x1  x2  x3  y
id                
297  43  59   2  1
298  67  66   0  1
299  58  60   3  1
300  49  63   3  1
301  45  60   0  1
302  54  58   1  1
303  56  66   3  1
304  42  69   1  1
305  50  59   2  1
306  59  60   0  1
